In [1]:
import sys
import torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
from _utils import  sample_random_glue_stsb, collect_info_for_metric, save_info, \
                    get_continuation_mapping, get_continuous_attributions, get_continuous_raw_inputs,\
                    attr_normalizing_func
from preload_models import get_stsb_tokenizer_n_model

In [2]:
stsb_data_raw, targets, idxs = sample_random_glue_stsb()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

[ 436  138  931 1462 1491  577  873  899  268  603 1341  648  941  901
  335 1306  234  132 1257 1433 1351 1484  977  146 1275  225  396  757
  949 1151  605 1041  350  788  626   88  599 1080  882  927 1209  647
  486  214  317  895  897   50 1054 1454]


In [3]:
tokenizer, model = get_stsb_tokenizer_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import GuidedBackprop
from captum.attr import visualization 

In [6]:
gbp  = GuidedBackprop(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    print(f'pred_prob {pred_prob*5}')

     #categorizing results
    pred_class = 'Similar' if pred_prob > 0.5 else 'Not Similar' 
    true_class = 'Similar' if target > 2.5 else 'Not Similar' 
    
    #attribution algorithm working
    attribution = gbp.attribute(input_emb)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Similar' if attr_score > 0.5 else 'Not Similar'
    convergence_score = None
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(stsb_data_raw, targets), start=1):
#     example_1 = 'The man cut down a tree with an axe.'
#     example_2 = 'A man chops down a tree with an axe.'
#     datum_raw, target = [example_1, example_1], 1
    print(f'Raw review: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))   

Raw review: ['Two dogs are sniffing something in the rocks.', 'Two dogs running down a path in the woods.']
GT target: 1.600000023841858
pred_prob 2.0290477573871613


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:60: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


word attr tensor([-0.0009, -0.0010, -0.5440, -0.0428, -0.0387,  0.0559, -0.0500,  0.0085,
        -0.1523,  0.0310,  0.0209, -0.0954, -0.0911, -0.0764, -0.5069, -0.2136,
        -0.1650,  0.1278, -0.0262,  0.0331, -0.0397,  0.5366, -0.0772,  0.0620])
conti attr [tensor(-0.0009), tensor(-0.0010), tensor(-0.5440), tensor(-0.0428), tensor(0.0172), tensor(-0.0500), tensor(0.0085), tensor(-0.1523), tensor(0.0518), tensor(-0.0954), tensor(-0.0911), tensor(-0.0764), tensor(-0.5069), tensor(-0.2136), tensor(-0.1650), tensor(0.1278), tensor(-0.0262), tensor(0.0331), tensor(-0.0397), tensor(0.4594), tensor(0.0620)]
detokenized ['<s>', 'Two', 'dogs', 'are', 'sniff', 'ing', 'something', 'in', 'the', 'rocks', '.', '</s>', '</s>', 'Two', 'dogs', 'running', 'down', 'a', 'path', 'in', 'the', 'woods', '.', '</s>']
len conti_raw 21
conti_raw ['<s>', 'Two', 'dogs', 'are', 'sniffing', 'something', 'in', 'the', 'rocks.', '</s>', '</s>', 'Two', 'dogs', 'running', 'down', 'a', 'path', 'in', 'the', 'woods.', 

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.41),Not Similar,-1.25,#s Two dogs are sniffing something in the rocks. #/s #/s Two dogs running down a path in the woods. #/s


<IPython.core.display.HTML object>
Raw review: ['A woman is slicing a potato.', 'A woman is slicing carrot.']
GT target: 2.5
pred_prob 2.1180687844753265
word attr tensor([-0.1151, -0.1078, -0.3735, -0.0316, -0.4018,  0.0620, -0.2990,  0.0686,
        -0.1297, -0.1099, -0.1268, -0.1140, -0.0391, -0.4891,  0.4972,  0.1501,
        -0.0824])
conti attr [tensor(-0.1151), tensor(-0.1078), tensor(-0.3735), tensor(-0.0316), tensor(-0.4018), tensor(0.0620), tensor(-0.2304), tensor(-0.1297), tensor(-0.1099), tensor(-0.1268), tensor(-0.1140), tensor(-0.0391), tensor(-0.4891), tensor(0.6472), tensor(-0.0824)]
detokenized ['<s>', 'A', 'woman', 'is', 'slicing', 'a', 'potato', '.', '</s>', '</s>', 'A', 'woman', 'is', 'slicing', 'carrot', '.', '</s>']
len conti_raw 15
conti_raw ['<s>', 'A', 'woman', 'is', 'slicing', 'a', 'potato.', '</s>', '</s>', 'A', 'woman', 'is', 'slicing', 'carrot.', '</s>']


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.14),Similar,2.97,#s Two little girls are talking on the phone. #/s #/s A little girl is walking down the street. #/s


<IPython.core.display.HTML object>
Raw review: ["The report shows that drugs sold in Canadian pharmacies are manufactured in facilities approved by Health Canada - the FDA's counterpart in Canada.", 'The report shows that drugs sold in Canadian pharmacies are manufactured in facilities approved by Health Canada, which serves a similar role as the FDA for the Canadian government.']
GT target: 5.0
pred_prob 4.7438859939575195
word attr tensor([-8.5463e-02,  1.9276e-02,  4.3818e-04, -7.6164e-02, -7.2822e-03,
        -4.4617e-02, -6.3596e-02, -7.9740e-02,  2.6286e-03,  8.6366e-03,
        -1.1602e-01, -7.7361e-02, -5.8177e-02, -1.2019e-01,  1.2158e-02,
        -3.9355e-03, -2.2899e-01, -2.1536e-02, -3.6674e-02,  4.7354e-02,
         5.3167e-01, -3.4148e-01,  1.0075e-01, -3.8989e-02, -2.7266e-02,
        -3.1227e-02, -5.3406e-02, -3.2050e-02, -2.3151e-02,  4.6530e-02,
        -7.0584e-02, -3.3355e-04, -2.6807e-02, -5.5083e-03, -7.9750e-02,
        -4.1476e-02,  3.6016e-02, -6.3734e-02,  2.9

<IPython.core.display.HTML object>
Raw review: ["Foreign companies eye new 'opening-up'", 'More carriers likely in China']
GT target: 0.6000000238418579
pred_prob 0.07349737454205751
word attr tensor([ 0.2237,  0.0900, -0.0916,  0.1471, -0.1415,  0.0080,  0.1642, -0.2788,
        -0.4668,  0.0763, -0.0066,  0.0742, -0.0732, -0.4660, -0.1280,  0.0154,
         0.5411, -0.1608])
conti attr [tensor(0.2237), tensor(0.0900), tensor(-0.0916), tensor(0.1471), tensor(-0.1415), tensor(-0.4971), tensor(-0.0066), tensor(0.0742), tensor(-0.0732), tensor(-0.4660), tensor(-0.1280), tensor(0.0154), tensor(0.5411), tensor(-0.1608)]
detokenized ['<s>', 'Foreign', 'companies', 'eye', 'new', "'", 'opening', '-', 'up', "'", '</s>', '</s>', 'More', 'carriers', 'likely', 'in', 'China', '</s>']
len conti_raw 14
conti_raw ['<s>', 'Foreign', 'companies', 'eye', 'new', "'opening-up'", '</s>', '</s>', 'More', 'carriers', 'likely', 'in', 'China', '</s>']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.01),Not Similar,-0.47,#s Foreign companies eye new 'opening-up' #/s #/s More carriers likely in China #/s


<IPython.core.display.HTML object>


In [9]:
save_info(idxs, stsb_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='guided_backprop_out.pkl')

{'indices': [436,
  138,
  931,
  1462,
  1491,
  577,
  873,
  899,
  268,
  603,
  1341,
  648,
  941,
  901,
  335,
  1306,
  234,
  132,
  1257,
  1433,
  1351,
  1484,
  977,
  146,
  1275,
  225,
  396,
  757,
  949,
  1151,
  605,
  1041,
  350,
  788,
  626,
  88,
  599,
  1080,
  882,
  927,
  1209,
  647,
  486,
  214,
  317,
  895,
  897,
  50,
  1054,
  1454],
 'raw_data': [['Two dogs are sniffing something in the rocks.',
   'Two dogs running down a path in the woods.'],
  ['A woman is slicing a potato.', 'A woman is slicing carrot.'],
  ['It was before the season began because Amy saw the woman before the Silence took her.',
   "The exact timeline hasn't been listed yet, but there are a few assumptions we can make."],
  ['Protests for kidnapped girls banned in Nigerian capital',
   "Scores 'killed' in Boko Haram raid in Nigeria"],
  ['Oil falls in Asian trade', 'Oil prices down in Asian trade'],
  ['There are people out on the street.', 'People are out on the street.'],
 